In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv("C:/Users/anies/Downloads/Bank_Personal_Loan_Modelling.csv")

data.head()

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1


In [2]:
data = data.drop(columns = ["ID","ZIP Code"])
X = data.drop("Personal Loan",axis=1)


In [3]:
y = data["Personal Loan"]
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

In [4]:


X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=0)
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [5]:

import keras
from keras.models import Sequential
from keras.layers import Dense

from keras.optimizers import Adam

optimizer = Adam(lr=0.01)
loss = 'binary_crossentropy'

C:\Users\anies\anaconda3\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [6]:
def fitness(weights):
    w0 = weights[:66].reshape((11, 6))
    w1 = weights[66:72].reshape((6,1))
    b0= np.array([0.,0.,0.,0.,0.,0.])
    b1 = np.array([0.])


    model.layers[0].set_weights([w0, b0])
    model.layers[1].set_weights([w1, b1])
    model.compile(optimizer=optimizer, loss=loss)
    Loss = model.evaluate(X_train, y_train, verbose=0)
    return -Loss

In [7]:
def genetic_algorithm(population_size, generations, mutation_rate):

    population = [np.random.uniform(low=-1, high=1, size=77) for _ in range(population_size)]
    
    for generation in range(generations):
        fitness_scores = [fitness(x) for x in population]

        indices = np.argsort(fitness_scores)[-2:]
        parents = [population[i] for i in indices]

        offspring = []
        for _ in range(population_size - len(parents)):
            parent1 = parents[0].flatten()
            parent2 = parents[1].flatten()

            crossover_point = np.random.randint(0, len(parent1))
            child = np.concatenate([parent1[:crossover_point], parent2[crossover_point:]])
            child = child.reshape(parents[0].shape)

            for i in range(len(child)):
                if np.random.uniform() < mutation_rate:
                    child[i] += np.random.normal(loc=0, scale=0.1)

            offspring.append(child)

        population = parents + offspring
        
        if (generation%5==0):
            print("Finished Generation:",generation)

    fitness_scores = [fitness(x) for x in population]
    best_index = np.argmax(fitness_scores)
    best_weights = population[best_index]

    return best_weights

In [8]:
model = Sequential()
model.add(Dense(units=6, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))

model.weights

[<tf.Variable 'dense/kernel:0' shape=(11, 6) dtype=float32, numpy=
 array([[-0.4389543 ,  0.57338405, -0.14651254,  0.19232213, -0.4666832 ,
          0.29054093],
        [ 0.49394143,  0.49870372, -0.39886373, -0.5399896 ,  0.52930796,
         -0.4744595 ],
        [ 0.5935787 ,  0.4724859 , -0.3012084 , -0.35549062,  0.12195611,
          0.3958506 ],
        [ 0.18018997,  0.4048519 ,  0.3530333 , -0.59275883, -0.13992122,
         -0.12276885],
        [ 0.556551  ,  0.46395934,  0.30569535, -0.04689008, -0.11823601,
          0.21897078],
        [ 0.58351827,  0.59052885,  0.23687273,  0.11033964, -0.47696117,
          0.52959096],
        [-0.31725612,  0.5834806 , -0.39892492,  0.373026  , -0.02793843,
          0.2505433 ],
        [-0.49324033, -0.2910347 , -0.21713948, -0.04087681, -0.20238835,
         -0.51177037],
        [ 0.01818186, -0.3107328 ,  0.5659548 , -0.4849775 , -0.02247447,
          0.06060284],
        [ 0.2086097 , -0.24125698, -0.1461995 ,  0.32997388,

In [9]:
best_weights = genetic_algorithm(population_size=20, generations=50, mutation_rate=0.1)
w0 = best_weights[:66].reshape((11, 6))
w1 = best_weights[66:72].reshape((6,1))
b0= np.array([0.,0.,0.,0.,0.,0.])
b1 = np.array([0.])

model.layers[0].set_weights([w0, b0])
model.layers[1].set_weights([w1, b1])

model.compile(optimizer=optimizer, loss=loss)

Finished Generation: 0
Finished Generation: 5
Finished Generation: 10
Finished Generation: 15
Finished Generation: 20
Finished Generation: 25
Finished Generation: 30
Finished Generation: 35
Finished Generation: 40
Finished Generation: 45


In [10]:
y_pred = model.predict(X_test)
y_pred = (y_pred < 0.00005)

47/47 [==============================] - 0s 874us/step


In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.914